## Creating short movies to verify emission and state probability
By adding this notebook in the workflow, you can create movies of emission probability together with state probability function.  The notebook is setted that it creates short movie (once in 24 time step). 
You can adjust it to have long movie by adjusting time_step=1. 

In [ ]:
import dask
from distributed import LocalCluster

In [ ]:
cluster = LocalCluster(n_workers=1)#, threads_per_worker=1)
client = cluster.get_client()
client

In [ ]:
import warnings

import matplotlib.pyplot as plt
import xarray as xr
from shapely.errors import ShapelyDeprecationWarning
from xmovie import Movie

warnings.filterwarnings(
    action="ignore",
    category=ShapelyDeprecationWarning,  # in cartopy
)
warnings.filterwarnings(
    action="ignore",
    category=UserWarning,
    message=r"No `(vmin|vmax)` provided. Data limits are calculated from input. Depending on the input this can take long. Pass `\1` to avoid this step",
)
%matplotlib inline

In [ ]:

# with acoustic
root = "file:///Users/todaka/python/git/pangeo-fish/data_local/fish-intel/A18832-f1_e2500-hp4096-"

emission_path = "-acoustic_tina"
states_path = "-acoustic_tina_bugfix"
path_e = f"{root}emission{emission_path}.zarr"
path = f"{root}state{states_path}.zarr"

# without acoustic
root = "file:///Users/todaka/python/git/pangeo-fish/data_local/fish-intel/A18832-f1_e2500-hp4096-"

emission_path = "tina"
states_path = "tina_new_bugfix"
path_e = f"{root}emission_{emission_path}.zarr"
path = f"{root}state_{states_path}.zarr"

time_step = 6
path

In [ ]:
ds_e = xr.open_dataset(path_e, engine="zarr", chunks={"x": -1, "y": -1})
ds = xr.open_dataset(path, engine="zarr", chunks={"x": -1, "y": -1})
ds["pdf"] = ds_e.pdf

In [ ]:
def drop_0_subset(ds):
    cond = ds.states.sum(dim="time", skipna=True).compute() > 0.0
    return ds.where(cond, drop=True)


ds_subset = ds.pipe(drop_0_subset)

In [ ]:
bbox = (
    ds_subset.longitude.min(),
    ds_subset.latitude.min(),
    ds_subset.longitude.max(),
    ds_subset.latitude.max(),
)


def geo_subset(ds, bbox):
    x0, y0, x1, y1 = bbox
    cond = (
        (ds.longitude.compute() >= x0)
        & (ds.longitude.compute() <= x1)
        & (ds.latitude.compute() >= y0)
        & (ds.latitude.compute() <= y1)
    )

    return ds.where(cond, drop=True)


ds = ds_subset.pipe(geo_subset, bbox)
ds  # ["pdf"] =
# ds_e.pipe(geo_subset, bbox).pdf

In [ ]:
def custom_plotfunc(ds, fig, tt, *args, **kwargs):
    import warnings

    import cartopy.crs as ccrs
    import cartopy.feature as cf
    import cmocean
    import matplotlib.pyplot as plt
    from shapely.errors import ShapelyDeprecationWarning

    warnings.filterwarnings(
        action="ignore",
        category=ShapelyDeprecationWarning,  # in cartopy
    )
    warnings.filterwarnings(
        action="ignore",
        category=UserWarning,
        message=r"No `(vmin|vmax)` provided. Data limits are calculated from input. Depending on the input this can take long. Pass `\1` to avoid this step",
    )
    %matplotlib inline

    projection = ccrs.Mercator()
    crs = ccrs.PlateCarree()
    bbox = (
        ds.longitude.min(),
        ds.latitude.min(),
        ds.longitude.max(),
        ds.latitude.max(),
    )
    x0, y0, x1, y1 = bbox

    (ax, ax2) = fig.subplots(
        ncols=2,
        # sharex=True,
        # sharey=True,
        subplot_kw=dict(projection=projection, frameon=True),
    )

    ax.add_feature(cf.COASTLINE.with_scale("10m"), lw=0.5)
    ax.add_feature(cf.BORDERS.with_scale("10m"), lw=0.3)
    ax.set_extent([x0, x1, y0, y1], crs=crs)
    gl = ax.gridlines(
        crs=crs,
        draw_labels=True,
        linewidth=0.6,
        color="gray",
        alpha=0.5,
        linestyle="-.",
    )
    gl.xlabel_style = {"size": 15}
    gl.ylabel_style = {"size": 15}

    cbar_kwargs = {
        "orientation": "horizontal",
        "shrink": 0.9,
        "pad": 0.1,
        "aspect": 80,
        "label": "State Probability",
    }

    ds.states.isel(time=tt).plot(
        ax=ax,
        x="longitude",
        y="latitude",
        cbar_kwargs=cbar_kwargs,
        transform=ccrs.PlateCarree(),
        # vmin=0,
        # vmax=0.00001,
        cmap="cool",
    )

    ax2.add_feature(cf.COASTLINE.with_scale("10m"), lw=0.5)
    ax2.add_feature(cf.BORDERS.with_scale("10m"), lw=0.3)
    ax2.set_extent([x0, x1, y0, y1], crs=crs)

    gl2 = ax2.gridlines(
        crs=crs,
        draw_labels=True,
        linewidth=0.6,
        color="gray",
        alpha=0.5,
        linestyle="-.",
    )
    gl2.xlabel_style = {"size": 15}
    gl2.ylabel_style = {"size": 15}

    cbar_kwargs = {
        "orientation": "horizontal",
        "shrink": 0.9,
        "pad": 0.1,
        "aspect": 80,
        "label": "Emission Probability",
    }

    ds.pdf.isel(time=tt).plot(
        ax=ax2,
        x="longitude",
        y="latitude",
        cbar_kwargs=cbar_kwargs,
        transform=ccrs.PlateCarree(),
        # vmin=0,
        # vmax=0.0001,
        cmap="cool",
    )

    return None, None

In [ ]:
%%time

mov = Movie(
    ds.isel(time=slice(0, ds.time.size + 1, time_step)),
    custom_plotfunc,
    input_check=False,
)

In [ ]:
%%time
# mov.preview(1)
path = f"{name}_{states_path}_{time_step}hours.mp4"
mov.save(path, progress=True, overwrite_existing=True)
# mov.render_single_frame(10)

In [ ]:
!rm -rf *.png